In [1]:
import os
import numpy as np
import pandas as pd
import rouge
import evaluate

from helpers import *

/usr/local/Caskroom/miniconda/base/envs/1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")

In [37]:
def insert_row_at_index(df, specific_df, row_index):
    if row_index is None:
        return pd.concat([df, specific_df])

    for col in specific_df.columns:
        df.at[row_index, col] = specific_df[col].values[0]

    return df

In [38]:
def update_common_df(evaluation_directory, specific_df, approach_name, method_name):
    common_df_path = os.path.join(evaluation_directory, "common.csv")

    if os.path.exists(common_df_path):
        common_df = pd.read_csv(common_df_path)
        query_string = f"Approach == '{approach_name}' and Method == '{method_name}'"
        matching_row = common_df.query(query_string)
        row_index = matching_row.index[0] if not matching_row.empty else None
        common_df = common_df.drop(matching_row.index)
        common_df = insert_row_at_index(common_df, specific_df, row_index)
        common_df = common_df.sort_index()
    else:
        common_df = specific_df

    common_df.to_csv(common_df_path, index=False)

In [39]:
def save_evaluation_to_df(rouge_score, bert_score, language, approach_name, model_name):
    specific_df = pd.DataFrame([{
        'Approach': approach_name,
        'Method': model_name,
        'Language': language,
        **rouge_score,
        **bert_score
    }])

    evaluation_folder = os.path.join(os.path.curdir, "evaluated")
    language_folder = os.path.join(evaluation_folder, language)
    create_folder_if_not_exists(language_folder)
    method_folder = os.path.join(language_folder, approach_name)
    create_folder_if_not_exists(method_folder)
    evaluation_row_file_name = os.path.join(method_folder, f"{model_name}.csv")
    specific_df.to_csv(evaluation_row_file_name, index=False)
    update_common_df(language_folder, specific_df, approach_name, model_name)

In [40]:
def rouge_evaluate(references, predictions):
    rouge_types = ['rouge1', 'rouge2', 'rougeL']
    results = rouge.compute(predictions=predictions, references=references, rouge_types=rouge_types)
    rouge_1 = round(results['rouge1'], 2)
    rouge_2 = round(results['rouge2'], 2)
    rouge_L = round(results['rougeL'], 2)

    avg_rouge = round(np.mean([rouge_1, rouge_2, rouge_L]), 2)

    return {"rouge_1": rouge_1, "rouge_2": rouge_2, "rouge_L": rouge_L, "avg_rouge": avg_rouge}

In [41]:
def bertscore_evaluate(references, predictions):
    results = bertscore.compute(predictions=predictions, references=references, lang="en")
    precision = results["precision"]
    recall = results["recall"]
    f1 = results["f1"]

    avg_precision = round(np.mean(precision), 2)
    avg_recall = round(np.mean(recall), 2)
    avg_f1 = round(np.mean(f1), 2)

    return {"precision": avg_precision, "recall": avg_recall, "f1": avg_f1}

In [42]:
def evaluate_models(language, language_folder, summarization_method_name, single_model, references):
    method_folder = os.path.join(language_folder, summarization_method_name)

    models = os.listdir(method_folder)

    for _, model_name in enumerate(models):
        if single_model != "all" and model_name != single_model:
            continue

        model_folder = os.path.join(method_folder, model_name)
        summaries = os.listdir(model_folder)
        predictions = []

        for summary_index, _ in enumerate(summaries):
            summary_file = os.path.join(model_folder, f"{summary_index}.txt")

            with open(summary_file, 'r') as file:
                generated_summary_text = file.read()
                predictions.append(generated_summary_text)

        bert_score = bertscore_evaluate(references, predictions)
        rouge_score = rouge_evaluate(references, predictions)
        save_evaluation_to_df(rouge_score, bert_score, language, summarization_method_name, model_name)

In [44]:
def evaluate_generated_summaries(language, single_approach="all", single_model="all"):
    storage_folder = os.path.join(os.path.curdir, "laws_with_abstracts")
    summarized_folder = os.path.join(storage_folder, "summarized")
    language_folder = os.path.join(summarized_folder, language)
    summarized_methods_listed = os.listdir(language_folder)
    references = get_references(language, "abstracts")

    for _, summarization_method_name in enumerate(summarized_methods_listed):
        if single_approach != "all" and summarization_method_name != single_approach:
            continue

        evaluate_models(language, language_folder, summarization_method_name, single_model, references)

In [45]:
evaluate_generated_summaries("ukrainian")
# evaluate_generated_summaries("english", "zero_shot_abstractive", "gpt2_xl")

In [47]:
references = get_references("ukrainian", "abstracts")
references

['Статтею 1 Закону встановлено наступне: період проведення антитерористичної операції – це час між датою набрання чинності Указом Президента України "Про рішення Ради національної безпеки і оборони України від 13 квітня 2014 року "Про невідкладні заходи щодо подолання терористичної загрози і збереження територіальної цілісності України" від 14 квітня 2014 року № 405/2014 та датою набрання чинності Указом Президента України про завершення проведення антитерористичної операції або військових дій на території України;\nтериторія проведення антитерористичної операції – це територія України, на якій розташовані населені пункти, визначені у затвердженому Кабінетом Міністрів України переліку, де проводилася антитерористична операція, розпочата відповідно до Указу Президента України "Про рішення Ради національної безпеки і оборони України від 13 квітня 2014 року "Про невідкладні заходи щодо подолання терористичної загрози і збереження територіальної цілісності України" від 14 квітня 2014 року 